# Langserve Backend Endpoint 

> A server-side delivery of langchain agents and chains (based on externally defined `add_routes()` function calls.)

Run this code as the following:

```
python -m alhazen.backend 
```

The system will include additional endpoints defined by subclasses.

In [7]:
#| default_exp backend

In [8]:
#| export
import asyncio
from uuid import UUID

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from langchain.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.globals import set_debug
from langchain.schema.embeddings import Embeddings
from langchain.schema.retriever import BaseRetriever
from langchain.vectorstores.pgvector import PGVector

from langserve import add_routes

from langsmith import Client

import os
from operator import itemgetter
from pydantic import BaseModel

# Hack to fix Incorrect formatting for Llama Chat models (including Mixtral)
from alhazen.utils.langchain_utils import ChatPromptValue_to_string
from langchain.prompts.chat import ChatPromptValue
ChatPromptValue.to_string = ChatPromptValue_to_string

from alhazen.langserve.chat import ChatRequest, get_chain_for_app

In [10]:
#| export 

def get_embeddings_model() -> Embeddings:
    model_name = "BAAI/bge-large-en"
    model_kwargs = {"device": "mps"}
    encode_kwargs = {"normalize_embeddings": True}
    return HuggingFaceBgeEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )

def get_retriever() -> BaseRetriever:    
    pgvector_docstore = PGVector.from_existing_index(
        embedding=get_embeddings_model(),
        collection_name='ScienceKnowledgeItem_CitationRecord')
    return pgvector_docstore.as_retriever(search_kwargs=dict(k=6))

def get_llm() -> BaseRetriever:
    llm = ChatOllama(model="mixtral", streaming=True, temperature=0.1)
    return llm

'''class SendFeedbackBody(BaseModel):
    run_id: UUID
    key: str = "user_score"

    score: Union[float, int, bool, None] = None
    feedback_id: Optional[UUID] = None
    comment: Optional[str] = None


@app.post("/feedback")
async def send_feedback(body: SendFeedbackBody):
    client.create_feedback(
        body.run_id,
        body.key,
        score=body.score,
        comment=body.comment,
        feedback_id=body.feedback_id,
    )
    return {"result": "posted feedback successfully", "code": 200}


class UpdateFeedbackBody(BaseModel):
    feedback_id: UUID
    score: Union[float, int, bool, None] = None
    comment: Optional[str] = None


@app.patch("/feedback")
async def update_feedback(body: UpdateFeedbackBody):
    feedback_id = body.feedback_id
    if feedback_id is None:
        return {
            "result": "No feedback ID provided",
            "code": 400,
        }
    client.update_feedback(
        feedback_id,
        score=body.score,
        comment=body.comment,
    )
    return {"result": "patched feedback successfully", "code": 200}
'''

# TODO: Update when async API is available
async def _arun(func, *args, **kwargs):
    return await asyncio.get_running_loop().run_in_executor(None, func, *args, **kwargs)

async def aget_trace_url(run_id: str) -> str:
    for i in range(5):
        try:
            await _arun(client.read_run, run_id)
            break
        except langsmith.utils.LangSmithError:
            await asyncio.sleep(1**i)

    if await _arun(client.run_is_shared, run_id):
        return await _arun(client.read_run_shared_link, run_id)
    return await _arun(client.share_run, run_id)

class GetTraceBody(BaseModel):
    run_id: UUID

In [ ]:
#| export

app = FastAPI(
    title="Alhazen Server",
    version="0.0.1",
    description="An api server using Langserve to support the Alhazen agent's tools.",
)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

@app.post("/get_trace")
async def get_trace(body: GetTraceBody):
    run_id = body.run_id
    if run_id is None:
        return {
            "result": "No LangSmith run ID provided",
            "code": 400,
        }
    return await aget_trace_url(str(run_id))

if os.environ.get('ALHAZEN_DB_NAME') is not None:
    os.environ['PGVECTOR_CONNECTION_STRING'] = "postgresql+psycopg2://localhost:5432/"+os.environ['ALHAZEN_DB_NAME']
    
    client = Client()

    # Adds routes to the app for using the chain under:
    # /invoke
    # /batch
    # /stream
    add_routes(app, \
            get_chain_for_app(get_llm(), get_retriever()), \
            path="/chat", \
            input_type=ChatRequest, \
            config_keys=["metadata"])

In [1]:
#| export 
#| output: false

if __name__ == "__main__" and os.environ.get('ALHAZEN_DB_NAME') is not None:
    import uvicorn
    uvicorn.run(app, host="localhost", port=8080)
else: 
    print('Please set `ALHAZEN_DB_NAME` to name of locally-installed PostGresQL database to be used with this chat client.')

NameError: name 'os' is not defined